# Alliance log parsing

Import some basic packages

In [42]:
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

In [2]:
from tqdm import tqdm_notebook as tqdm

In [3]:
import pynmea2

In [4]:
import mypaths

In [5]:
date = datetime(2018, 2, 23)

In [6]:
time_range = pd.date_range(start=date, freq='S', periods=86400).to_series().to_frame(name='time')
# time_range.index = pd.Index(range(86400))

In [7]:
fname = mypaths.igp_data_dir / 'nmea_logs' / f'{date:%Y%m%d}.log'

In [31]:
ggas = {'datetime': [], 'timestamp': [], 'lon': [], 'lat': []}
hdts = {'datetime': [], 'heading': []}
inmwv = {'datetime': [], 'wind_speed': [], 'wind_angle': [], 'status': [], 'reference': []}
with fname.open('r') as f:
    for line in tqdm(f.readlines()):
        try:
            msg = pynmea2.NMEASentence.parse(line)
            if isinstance(msg, pynmea2.talker.GGA):
#                 d1 = datetime.combine(date, msg.timestamp)
#                 d2 = datetime.strptime(msg.datetime_str, '%Y%m%d%H%M%S')
                
#                 if d1 != d2:
#                     print(msg)
                
                ggas['datetime'].append(msg.datetime_str)
                ggas['timestamp'].append(datetime.combine(date, msg.timestamp))
                ggas['lon'].append(msg.longitude)
                ggas['lat'].append(msg.latitude)
            elif isinstance(msg, pynmea2.talker.HDT):
                hdts['datetime'].append(msg.datetime_str)
                hdts['heading'].append(float(msg.heading))
            elif isinstance(msg, pynmea2.talker.MWV):
                inmwv['datetime'].append(msg.datetime_str)
                inmwv['wind_speed'].append(float(msg.wind_speed))
                inmwv['wind_angle'].append(float(msg.wind_angle))
                inmwv['status'].append(str(msg.status))
                inmwv['reference'].append(str(msg.reference))
        except pynmea2.ParseError:
            pass

In [32]:
len(inmwv['datetime']), len(ggas['datetime']), len(hdts['datetime'])

(86397, 86412, 86401)

In [55]:
gga_df = pd.DataFrame(data=ggas)
gga_df['datetime'] = gga_df['datetime'].astype(int).apply(lambda x: datetime(1970, 1, 1) + timedelta(seconds=x))

In [56]:
hdt_df = pd.DataFrame(data=hdts)
hdt_df['datetime'] = hdt_df['datetime'].astype(int).apply(lambda x: datetime(1970, 1, 1) + timedelta(seconds=x))
# hdt_df['heading'] = hdt_df['heading'].astype(float)

In [57]:
inmwv_df = pd.DataFrame(data=inmwv)
inmwv_df['datetime'] = inmwv_df['datetime'].astype(int).apply(lambda x: datetime(1970, 1, 1) + timedelta(seconds=x))
# inmwv_df['wind_speed'] = inmwv_df['wind_speed'].astype(float)
# inmwv_df['wind_angle'] = inmwv_df['wind_angle'].astype(float)
# inmwv_df['status'] = inmwv_df['status'].astype(str)

In [58]:
# pd.merge(time_range, inmwv_df[inmwv_df.reference!='R'].drop_duplicates('datetime').set_index('datetime'),
#                    how='outer', left_index=True, right_index=True)

In [59]:
dataframes = [gga_df, inmwv_df, hdt_df]

In [61]:
df0 = time_range
for df in dataframes:
    df0 = pd.merge(df0, df.drop_duplicates('datetime').set_index('datetime'),
                   how='outer', left_index=True, right_index=True)
df0 = df0.drop('time', axis=1).drop('timestamp', axis=1)